In [1]:
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error as mae

`gms_all` holds all match data, `gms_10` holds only 2010 data, `gms_14` holds on 2014 data and `gms_exc_14` holds all data excluding the 2014 match data

In [2]:
gms_all = pd.read_csv('matches.csv')
gms_10 = gms_all.loc[gms_all['key']=='world.2010']
gms_14 = gms_all.loc[gms_all['key']=='world.2014']
gms_exc_14 = gms_all[gms_all['key']!='world.2014']
display(gms_10.describe())
display(gms_14.describe())

,id,round_id,score1,score2
count,64.000000,64.000000,64.00000,64.000000
mean,1606.500000,391.687500,1.18750,1.046875
std,18.618987,5.279114,1.34371,0.966579
min,1575.000000,382.000000,0.00000,0.000000
25%,1590.750000,387.000000,0.00000,0.000000
50%,1606.500000,392.500000,1.00000,1.000000
75%,1622.250000,396.250000,2.00000,1.250000
max,1638.000000,401.000000,7.00000,4.000000


,id,round_id,score1,score2
count,64.000000,64.000000,64.000000,64.000000
mean,834.500000,121.718750,1.359375,1.531250
std,18.618987,5.226516,1.186795,1.583333
min,803.000000,112.000000,0.000000,0.000000
25%,818.750000,117.000000,0.000000,0.000000
50%,834.500000,122.500000,1.000000,1.000000
75%,850.250000,126.250000,2.000000,3.000000
max,866.000000,131.000000,5.000000,7.000000


It's necessary to 'mirror' all the matches due to symmetry before fitting the model

In [3]:
gls_10 = pd.concat([gms_10.rename(columns={'score1':'score'}),
                    gms_10.rename(columns={'score2':'score','team1':'team2','team2':'team1'})])
poisson_model = smf.glm(formula="score ~ team1 + team2", data=gls_10, 
                        family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  score   No. Observations:                  128
Model:                            GLM   Df Residuals:                       65
Model Family:                 Poisson   Df Model:                           62
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -136.98
Date:                Tue, 19 Jun 2018   Deviance:                       76.045
Time:                        22:49:43   Pearson chi2:                     66.7
No. Iterations:                    21                                         
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -22.2783   1.69e+04     -0.001      0.999   -3.31e+04    3.31e+04
team1[T.Argentina]        22.0959   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Australia]        22.4852   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Brazil]           22.4561   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Cameroon]         21.7755   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Chile]            22.4197   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Côte d'Ivoire]    22.1038   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Denmark]          22.3803   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.England]          21.4962   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.France]           20.2928   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Germany]          22.9326   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Ghana]            21.6323   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Greece]           20.9006   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Honduras]          1.0206   2.31e+04   4.42e-05      1.000   -4.53e+04    4.53e+04
team1[T.Italy]            22.6408   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Japan]            22.1368   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Mexico]           21.4688   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Netherlands]      22.6389   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.New Zealand]      21.5048   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Nigeria]          21.3308   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.North Korea]      21.5388   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Paraguay]         21.6884   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Portugal]         22.4582   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Serbia]           21.6839   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Slovakia]         22.5414   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Slovenia]         21.7745   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.South Africa]     21.5975   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.South Korea]      21.9451   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Spain]            22.5100   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Switzerland]      21.1111   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.United States]    22.2145   1.69e+04      0.001      0.999   -3.31e+04    3.31e+04
team1[T.Uruguay]          22.0332   1.69e+04      0.001      0.999  

Now define the wrapper function for the predictions. In the event of previously unseen teams, resort to using the arithmetic means.

In [4]:
pd.set_option('mode.chained_assignment', None)
def pred_team_gls(model, team1, team2, gls_df, have_teams=None):
    if have_teams is None: #initialises when first run
        have_teams = set(gls_df['team1'])|set(gls_df['team2'])
    if team1 in have_teams: 
        if team2 in have_teams:
            pred = model.predict(pd.DataFrame(data={'team1': team1, 'team2': team2},index=[0])).values[0]
        else:
            team1_gls = pd.concat([gls_df.loc[gls_df['team1']==team1,'score1'],
                                gls_df.loc[gls_df['team2']==team1,'score2']])
            pred = team1_gls.mean()
    else:
        if team2 in have_teams:
            team2_gls = pd.concat([gls_df.loc[gls_df['team1']==team2,'score1'],
                                gls_df.loc[gls_df['team2']==team2,'score2']])
            pred = team2_gls.mean()        
        else:
            pred = gls_df['score'].mean()
    return np.round(pred,8)

gms_14['pred1'] = gms_14.apply(lambda r: pred_team_gls(poisson_model, r['team1'],r['team2'], gls_10), axis=1)
gms_14['pred2'] = gms_14.apply(lambda r: pred_team_gls(poisson_model, r['team2'],r['team1'], gls_10), axis=1)
pred_14_df = gms_14[['score1', 'score2', 'team1', 'team2', 'pred1', 'pred2']]
pred_14_df.head()

,score1,score2,team1,team2,pred1,pred2
0,3,1,Brazil,Croatia,0.750000,0.750000
1,1,0,Mexico,Cameroon,0.652688,1.338506
2,1,5,Spain,Netherlands,0.885424,0.436429
3,3,1,Chile,Australia,2.303845,2.222653
4,3,0,Colombia,Greece,1.000000,1.000000


In [5]:
def evaluate_predictions(pred_df, score1='score1', score2='score2',
                        pred1='pred1', pred2='pred2'):
    score1_mae = mae(gms_14[score1], gms_14[pred1])
    score2_mae = mae(gms_14[score2], gms_14[pred2])
    display('score1_mae is {}'.format(score1_mae))
    display('score2_mae is {}'.format(score2_mae))
    display('combined mae is {}'.format((score1_mae+score2_mae)/2))
    
evaluate_predictions(gms_14)

'score1_mae is 1.14221197'

'score2_mae is 1.3622858896875'

'combined mae is 1.25224892984375'

The Negative Binomial model offers a slight improvement over the Poisson

In [6]:
nbinom_model = smf.glm(formula="score ~ team1 + team2", data=gls_10, 
                        family=sm.families.NegativeBinomial()).fit()
gms_14.loc[:,'pred1'] = gms_14.apply(lambda r: pred_team_gls(nbinom_model, r['team1'],r['team2'], gls_10), axis=1)
gms_14.loc[:,'pred2'] = gms_14.apply(lambda r: pred_team_gls(nbinom_model, r['team2'],r['team1'], gls_10), axis=1)

evaluate_predictions(gms_14)

'score1_mae is 1.1256913885937498'

'score2_mae is 1.3099595985937498'

'combined mae is 1.2178254935937498'

Time to train on more data (excluding 2014) and see what the results look like

In [7]:
gls_exc_14 = pd.concat([gms_all.rename(columns={'score1':'score'}),
                    gms_all.rename(columns={'score2':'score','team1':'team2','team2':'team1'})])
regr = smf.glm(formula="score ~ team1 + team2", data=gls_exc_14, 
                        family=sm.families.Poisson()).fit()
display(regr.summary())
gms_14['pred1'] = gms_14.apply(lambda r: pred_team_gls(regr, r['team1'],r['team2'], gls_exc_14), axis=1)
gms_14['pred2'] = gms_14.apply(lambda r: pred_team_gls(regr, r['team2'],r['team1'], gls_exc_14), axis=1)

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  score   No. Observations:                 1672
Model:                            GLM   Df Residuals:                     1511
Model Family:                 Poisson   Df Model:                          160
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -2396.0
Date:                Tue, 19 Jun 2018   Deviance:                       1905.1
Time:                        22:49:45   Pearson chi2:                 1.72e+03
No. Iterations:                    22                                         
======================================================================================================
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -0.1814      0.384     -0.473      0.636      -0.933       0.570
team1[T.Angola]                       -1.1312      1.048     -1.079      0.280      -3.185       0.923
team1[T.Argentina]                     0.5931      0.308      1.925      0.054      -0.011       1.197
team1[T.Australia]                     0.1006      0.425      0.236      0.813      -0.733       0.934
team1[T.Austria]                       0.5487      0.335      1.637      0.102      -0.108       1.206
team1[T.Belgium]                       0.1816      0.329      0.552      0.581      -0.463       0.826
team1[T.Bolivia]                      -1.5993      1.043     -1.533      0.125      -3.644       0.445
team1[T.Bosnia-Herzegovina]            0.3111      0.593      0.524      0.600      -0.851       1.474
team1[T.Brazil]                        0.8788      0.302      2.906      0.004       0.286       1.471
team1[T.Bulgaria]                     -0.0505      0.365     -0.138      0.890      -0.766       0.665
team1[T.Cameroon]                     -0.1771      0.387     -0.458      0.647      -0.935       0.581
team1[T.Canada]                      -22.1680   2.48e+04     -0.001      0.999   -4.87e+04    4.86e+04
team1[T.Chile]                         0.4262      0.335      1.274      0.203      -0.230       1.082
team1[T.China]                       -22.3108   2.47e+04     -0.001      0.999   -4.85e+04    4.84e+04
team1[T.Colombia]                      0.3547      0.361      0.984      0.325      -0.352       1.061
team1[T.Costa Rica]                    0.6277      0.364      1.725      0.084      -0.085       1.341
team1[T.Croatia]                       0.3561      0.373      0.954      0.340      -0.376       1.088
team1[T.Cuba]                          0.4302      0.598      0.720      0.472      -0.741       1.601
team1[T.Czech Republic]                0.1628      0.656      0.248      0.804      -1.124       1.449
team1[T.Czechoslovakia (-1992)]        0.3659      0.340      1.076      0.282      -0.300       1.032
team1[T.Côte d'Ivoire]                 0.3233      0.413      0.783      0.434      -0.486       1.133
team1[T.Denmark]                       0.6723      0.355      1.893      0.058      -0.024       1.369
team1[T.Dutch East Indies (-1945)]   -22.5371   4.37e+04     -0.001      1.000   -8.57e+04    8.57e+04
team1[T.East Germany (-1989)]          0.0330      0.539      0.061      0.951      -1.023       1.089
team1[T.Ecuador]                       0.1979      0.436      0.454      0.650      -0.656       1.052
team1[T.Egypt]                         0.0482      0.654      0.074      0.941      -1.234       1.331
team1[T.El Salvador]                  -1.7021      1.044     -1.630      0.103      -3.748       0.344
team1[T.England]                       0.3669      0.316      1.160     

By using more training data, the prediction error goes down significantly

In [8]:
evaluate_predictions(gms_14)

'score1_mae is 0.9019667357812501'

'score2_mae is 1.0970985839062501'

'combined mae is 0.9995326598437502'

The Negative Binomial doesn't offer any improvement here

In [9]:
regr = smf.glm(formula="score ~ team1 + team2", data=gls_exc_14, 
                        family=sm.families.NegativeBinomial()).fit()
gms_14['pred1'] = gms_14.apply(lambda r: pred_team_gls(regr, r['team1'],r['team2'], gls_exc_14), axis=1)
gms_14['pred2'] = gms_14.apply(lambda r: pred_team_gls(regr, r['team2'],r['team1'], gls_exc_14), axis=1)
evaluate_predictions(gms_14)

'score1_mae is 0.90134692359375'

'score2_mae is 1.1005473404687498'

'combined mae is 1.00094713203125'

Now use the all the available World Cup to predict results for the 2018 matches

In [10]:
gls_all = pd.concat([gms_all.rename(columns={'score1':'score'}),
                    gms_all.rename(columns={'score2':'score','team1':'team2','team2':'team1'})])
pred_df = pd.read_csv('group_matches_2018_raw.csv')
regr_all = smf.glm(formula="score ~ team1 + team2", data=gls_all, 
                        family=sm.families.Poisson()).fit()
pred_df['pred1'] = gms_14.apply(lambda r: pred_team_gls(regr_all, r['team1'],r['team2'], gls_all), axis=1)
pred_df['pred2'] = gms_14.apply(lambda r: pred_team_gls(regr_all, r['team2'],r['team1'], gls_all), axis=1)
pred_df['pred1'] = np.round(pred_df['pred1'],0)
pred_df['pred2'] = np.round(pred_df['pred2'],0)
pred_df

,team1,team2,score1,score2,pred1,pred2
0,Russia,Saudi Arabia,5.0,0.0,2.0,1.0
1,Egypt,Uruguay,0.0,1.0,1.0,1.0
2,Russia,Egypt,NaN,NaN,1.0,2.0
3,Uruguay,Saudi Arabia,NaN,NaN,2.0,1.0
4,Uruguay,Russia,NaN,NaN,3.0,1.0
5,Saudi Arabia,Egypt,NaN,NaN,1.0,1.0
6,Morocco,Iran,0.0,1.0,2.0,2.0
7,Portugal,Spain,3.0,3.0,1.0,1.0
8,Portugal,Morocco,NaN,NaN,1.0,1.0
9,Iran,Spain,NaN,NaN,2.0,0.0
